In [1]:
import numpy as np
from numba import jit
import sys
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, fsolve
import networkx as nx
import graph_ensembles.methods as ge

%matplotlib inline

# Auxiliary Functions

In [2]:
def density_solver(p_fun, L, z0):
    return least_squares(lambda x: p_fun(x) - L, z0, bounds = (0,np.inf))

In [3]:
@jit(nopython=True)
def nonzero_strengths(strenghts):
    ind1,ind2 = strenghts.nonzero()
    aux_strengths = []
    for ii,jj in zip(ind1,ind2):
        aux_strengths.append(strenghts[ii,jj])
    return np.dstack((ind1,ind2,np.array(aux_strengths)))[0]

In [4]:
@jit(nopython=True)
def nonzero_strengths_block(strenghts, group_array):
    ind1,ind2 = strenghts.nonzero()
    aux_strengths = []
    aux_sect = []
    for ii,jj in zip(ind1,ind2):
        aux_sect.append(group_array[ii])
        aux_strengths.append(strenghts[ii,jj])
    return np.dstack((ind1, np.array(aux_sect), ind2, np.array(aux_strengths)))[0]

In [5]:
@jit(forceobj=True)
def dict_sect_to_array(dict_sect):
    """
    Function converting the dictionary mapping each node
    to the corresponding sector into a numpy array where 
    the node label correspond to the array index.

    Parameters
    ----------
    p: dict
        a dictionary that given the index of a node returns its group
    Returns
    -------
    np.ndarray
        an array mapping each node to its group
    """
    tmp_arr_sec = np.zeros(len(dict_sect.keys()),dtype=int)
    for key in dict_sect.keys():
        tmp_arr_sec[key] = dict_sect[key]
    return tmp_arr_sec

In [6]:
@jit(nopython=True)
def weights_for_stripe(out_strength, group_array):
    """
    Function computing the number of links going out
    from each sector.
    
    Parameters
    ----------
    out_strength: np.ndarray
        nonzero out strengths
    group_dict: dict
        a dictionary that given the index of a node returns its group
    weighted: bool
    Returns
    -------
    np.ndarray
        an array where each entry i is the number of going out links 
        from sector i.
    """

    sect = np.unique(group_array)
    n_sect = sect.shape[0]
    links_for_sect = np.zeros(n_sect)
    for i in np.arange(out_strength.shape[0]):
        sect_out = int(out_strength[i,1])
        s_out = out_strength[i,2]
        links_for_sect[sect_out] += s_out
    return(links_for_sect)


@jit(nopython=True)
def weights_for_block(out_strength,group_array):
    """
    Function computing the number of links going from
    one sector i to sector j for every possible couple i-j.

    Parameters
    ----------
    adj: np.ndarray
        the binary adjacency matrix
    group_dict: dict
        a dictionary that given the index of a node returns its group
    Returns
    -------
    np.ndarray
        an array where each entry i,j is the number of links going
        from sector i to sector j.
    """

    sect = np.unique(group_array)
    n_sect = sect.shape[0]
    links_for_sect = np.zeros(shape=(n_sect,n_sect))
    for i in np.arange(out_strength.shape[0]):
        ind_out = int(out_strength[i,0])
        sect_ind_out = int(out_strength[i,1])
        sect_out = int(out_strength[i,2])
        s_out = out_strength[i,3]
        links_for_sect[sect_ind_out,sect_out] += s_out
    return(links_for_sect)

In [7]:
@jit(nopython=True)
def links_for_stripe(adjacency, group_array):
    """
    Function computing the number of links for stripe.
    """
    sect = np.unique(group_array)
    n_sect = sect.shape[0]
    links_for_sect = np.zeros(n_sect)
    ind1,ind2 = adjacency.nonzero()
    for i, j in zip(ind1, ind2):
        sect_i = int(group_array[i])
        links_for_sect[sect_i] += 1
    return links_for_sect
    
    
@jit(nopython=True)
def links_for_block(adjacency,group_array):
    """
    Function computing the number of links for block.
    """
    sect = np.unique(group_array)
    n_sect = sect.shape[0]
    links_for_sect = np.zeros(shape=(n_sect,n_sect))
    ind1,ind2 = adjacency.nonzero()
    for i, j in zip(ind1, ind2):
        sect_i = int(group_array[i])
        sect_j = int(group_array[j])
        links_for_sect[sect_i,sect_j] += 1
    return links_for_sect

# Creating the Network

In [8]:
# Generate toplogy
hyp_dim = np.array([10,10,10,10,10])
n_of_sector = len(hyp_dim)
seed = 20
p=0.3

real_dim = []
adj_sectors = []
dim_list = [0]
avg_out_degree_random = []

for ii in range(n_of_sector):
    G = nx.fast_gnp_random_graph(hyp_dim[ii],p=p,directed=True)
    tmp_adj = nx.to_numpy_array(G)
    avg_out_degree_random.append(np.mean(G.out_degree))
    real_dim.append(tmp_adj.shape[0])
    dim_list.append(dim_list[ii]+real_dim[ii])
    adj_sectors.append(tmp_adj)
    
G = nx.fast_gnp_random_graph(n = np.sum(real_dim), p = 0.08, directed = True)
adj_aux_random = nx.to_numpy_array(G)


for ii in range(n_of_sector):
    
    adj_aux_random[dim_list[ii]:dim_list[ii+1]][:,dim_list[ii]:dim_list[ii+1]] = adj_sectors[ii]

tmp_ind = np.where((np.sum(adj_aux_random,axis=1) + np.sum(adj_aux_random,axis=0))>0)[0]
adj_aux_random = adj_aux_random[tmp_ind][:,tmp_ind]


# Mapping nodes to sector
dict_sectors = {}
for ii in range(n_of_sector):
    tmp = []
    for jj in range(dim_list[ii],dim_list[ii+1]):
        dict_sectors[jj] = ii

# Converting the dictionary to array
array_sectors = dict_sect_to_array(dict_sectors)
        
# Assigning Weights

adj = adj_aux_random
avg_degree = avg_out_degree_random

k_out = adj.sum(axis=1)
k_in = adj.sum(axis=0)
adj_weight = np.zeros_like(adj)


for ii in np.arange(adj.shape[0]):
    for jj in np.arange(adj.shape[0]):
        if ii!=jj:
            if adj[ii,jj]!=0:
                #adj_weight[ii,jj] = k_out[ii]*k_in[jj]
                tmp = int(k_out[ii]*k_in[jj]/(avg_degree[dict_sectors[ii]]))
                adj_weight[ii,jj] = np.random.uniform(low=np.min([5,tmp]),high=np.max([6,tmp]),size=1)

# Preparing the input data

### Strengths stripe

In [9]:
strengths_in_stripe = np.zeros(shape=(adj.shape[0],n_of_sector))
strengths_out_stripe = np.zeros(shape=(adj.shape[0],n_of_sector))

for ii in np.arange(n_of_sector):
    indices_1 = np.arange(dim_list[ii],dim_list[ii+1])
    strengths_out_stripe[indices_1,ii] = (adj_weight[indices_1,:]).sum(axis=1)
    for jj in np.arange(n_of_sector):
        indices_2 = np.arange(dim_list[jj],dim_list[jj+1])
        
        strengths_in_stripe[indices_1,jj] = (adj_weight[indices_2][:,indices_1]).sum(axis=0)

### Number of edges

In [10]:
L_tot = adj.sum()

### Nonzero Strengths

In [11]:
nz_out_strenght_stripe = nonzero_strengths(strengths_out_stripe)
nz_in_strenght_stripe = nonzero_strengths(strengths_in_stripe)

# Applying Methods

## Stripe Model one z

In [12]:
solution = density_solver(lambda x: ge.expected_links_stripe_one_z(
                                    nz_out_strenght_stripe,
                                    nz_in_strenght_stripe,
                                    x),
                                    L_tot,
                                    0.000003)
z_stripe_one = solution.x[0]
P_matrix_stripe_one = ge.prob_array_stripe_one_z(
                                    nz_out_strenght_stripe,
                                    nz_in_strenght_stripe,
                                    z_stripe_one,
                                    adj.shape[0],
                                    n_of_sector)
print((adj>0).sum(),P_matrix_stripe_one.sum())

280 279.99999999999994
